## Ideas para la ferretería con agentes de smolagent

NOTA: Recomendable ejecutarlo en un entorno virtual de python

In [ ]:
!pip install smolagents smolagents[litellm] python-dotenv

In [ ]:
!python.exe -m pip install --upgrade pip

### Cargar la clave de hugging face desde el fichero .env

In [1]:
import pprint

In [2]:
from dotenv import load_dotenv

load_dotenv()

True

### Ejemplo de tool para la ferretería

Descartar LiteLLMModel porque falla a la hora de finalizar en el step correcto.

In [34]:
'''
from smolagents import LiteLLMModel, CodeAgent, tool

# Configurar el modelo
model_id = "meta-llama/Llama-3.3-70B-Instruct"
model = LiteLLMModel(model_id='huggingface/'+model_id)
'''

Usar en su lugar HFApiModel que sí termina en el step adecuado.

In [3]:
from smolagents import HfApiModel, CodeAgent, tool

model_id = "meta-llama/Llama-3.3-70B-Instruct"
model = HfApiModel(model_id=model_id)

c:\Users\pobla\OneDrive\Escritorio\Agente\SMOL_AGENTES\Smol_Agent_Ferreteria\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Definir tools
@tool
def pregunta(mensaje: str) -> str:
    """
    Preguntar al usuario para obtener datos adicionales.

    Args:
        mensaje: Pregunta para el usuario.
    """
    respuesta_usuario = input(f"{mensaje}\nRespuesta: ")
    return respuesta_usuario

@tool
def respuesta_fuera_del_tema(mensaje: str) -> str:
    """
    Respuesta cuando las preguntas no están relacionadas con asuntos de la ferretería.

    Args:
        mensaje: Respuesta a enviar
    """
    return mensaje

@tool
def final_answer(mensaje: str) -> str:
# def respuesta_final(mensaje: str) -> str:
    """
    Respuesta final a la pregunta inicial y finalización del bucle.

    Args:
        mensaje: Texto de la respuesta.
    """
    return mensaje



# Configurar el agente con la tool de Fibonacci que acabamos de definir
agent = CodeAgent(tools=[pregunta, respuesta_fuera_del_tema, final_answer], model=model, add_base_tools=True)
agent.max_steps = 8

In [5]:
tools = ', '.join(agent.tools.keys())
prompt4 = f'''\nEres un asistente experto en ferretería que responde de manera directa y eficiente solamente a temas de ferretería y a ningún otro tema.

## Sigue estas reglas:

1. Si necesitas información, usa la tool pregunta() sin explicar por qué.
2. Cuando tengas la información, usa la tool final_answer().
3. Cuando la respuesta contiene una secuencia de pasos, indícalo como una lista numerada.
4. NO ejecutes más código después de final_answer().
5. UTILIZA solo las siguientes tools definidas y NINGUNA otra más:
{tools}
6. Responde siempre en castellano.

## Ejemplo de flujo correcto:
Usuario: "Necesito colgar un cuadro"
pregunta("¿Cuál es el tamaño del cuadro?")
Usuario responde: "1.5m x 1m"
pregunta("¿Qué tipo de pared es donde vas a colgar el cuadro?")
Usuario responde: "ladrillo"
final_answer("Para colgar un cuadro de 1.5m x 1m en una pared de ladrillo, usa tornillos para ladrillo o anclajes de expansión. Mide y marca la pared con precisión y elige un soporte adecuado al peso.")

## Avisos
1. Cuando la pregunta no esté relacionada con temas de la ferretería, SIEMPRE DEBES usar la tool final_answer() y usa como parámetro el siguiente texto:
"Solo respondo a preguntas relacionadas con temas de la ferretería, ¿en qué puedo ayudarle?"
2. Si respondes a una pregunta no relacionada con temas de ferretería tendrás una multa de $10,000,000
3. No definas nuevas funciones python.
'''

'''
1. NO generes "thoughts" ni explicaciones antes de llamar a una función.

NO hagas esto:
Thought: Necesito saber el tamaño...
Code: pregunta()
'''

def set_system_prompt(agent):
    prompt_templates = agent.prompt_templates
    # agent.prompt_templates['system_prompt'] += prompt4
    agent.prompt_templates['system_prompt'] = prompt4

set_system_prompt(agent)

In [ ]:
# pprint.pprint(agent.prompt_templates['system_prompt'])

La siguiente llamada debería fallar porque no es un tema de ferretería

In [6]:
result = agent.run(
    "Cuanto vale el término 10 de la serie de fibonacci",
)

print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Cuanto vale el término 10 de la serie de fibonacci                                                              │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Solo respondo a preguntas relacionadas con temas de la ferretería, ¿en qué puedo ayudarle?")       
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Solo respondo a preguntas relacionadas con temas de la ferretería, ¿en qué puedo ayudarle?

[Step 0: Duration 3.79 seconds| Input tokens: 464 | Output tokens: 31]

Solo respondo a preguntas relacionadas con temas de la ferretería, ¿en qué puedo ayudarle?


In [7]:
result = agent.run(
    "Necesito colgar un cuadro",
)

print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Necesito colgar un cuadro                                                                                       │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
Here is your code snippet:
¿Cuál es el tamaño del cuadro y qué tipo de pared es donde lo vas a colgar? 
Usaré esta información para darte una respuesta adecuada.
```pregunta("¿Cuál es el tamaño del cuadro y el tipo de pared donde lo vas a colgar?")```
Make sure to include code with the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_code>
Make sure to provide correct code blobs.

[Step 0: Duration 13.49 seconds| Input tokens: 460 | Output tokens: 64]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Para colgar un cuadro de 10x10 en una pared de yeso, es recomendable utilizar tornillos para yeso  
  o anclajes de expansión. Asegúrate de medir y marcar la pared con precisión y elige un soporte adecuado al peso  
  del cuadro.")                                                                                                    
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Para colgar un cuadro de 10x10 en una pared de yeso, es recomendable utilizar tornillos para 
yeso o anclajes de expansión. Asegúrate de medir y marcar la pared con precisión y elige un soporte adecuado al 
peso del cuadro.

[Step 4: Duration 13.61 seconds| Input tokens: 4,159 | Output tokens: 357]

Para colgar un cuadro de 10x10 en una pared de yeso, es recomendable utilizar tornillos para yeso o anclajes de expansión. Asegúrate de medir y marcar la pared con precisión y elige un soporte adecuado al peso del cuadro.


In [12]:
result = agent.run(
    "que herramientas estan de oferta?",
)

print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ que herramientas estan de oferta?                                                                               │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
Here is your code snippet:
Lo siento, pero no tengo acceso directo a la información actualizada sobre ofertas de herramientas. Sin embargo, 
puedo ofrecerte algunas sugerencias generales sobre dónde podrías encontrar ofertas de herramientas.

1. **Visita tiendas de ferretería en línea**: Muchas tiendas de ferretería online ofrecen descuentos y promociones 
regulares en herramientas de diversos tipos.
2. **Encuentra descuentos en supermercados**: Algunos supermercados tienen secciones de ferretería con ofertas 
especiales.
3. **Explora mercados de herramientas usadas**: Puedes encontrar herramientas a precios más bajos en mercados de 
segunda mano o en línea.
4. **Suscríbete a newsletters de tiendas**: Muchas tiendas de ferretería envían newsletters con ofertas y 
descuentos exclusivos a sus suscriptores.

Si necesitas encontrar ofertas específicas en herramientas para un proyecto o tarea determinada, puedo ayudarte a 
identificar las herramientas necesarias. ¿Cuál es el proyecto para el que necesitas herramientas?
Make sure to include code with the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_code>
Make sure to provide correct code blobs.

[Step 0: Duration 13.61 seconds| Input tokens: 459 | Output tokens: 243]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Puedes encontrar herramientas en oferta en línea en sitios como Amazon.com.mx, MercadoLibre, The   
  Home Depot México, Mercado Libre Mexico y Suministrointec, con descuentos de hasta 58% y envíos gratis.")        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Puedes encontrar herramientas en oferta en línea en sitios como Amazon.com.mx, MercadoLibre, 
The Home Depot México, Mercado Libre Mexico y Suministrointec, con descuentos de hasta 58% y envíos gratis.

[Step 4: Duration 4.99 seconds| Input tokens: 8,423 | Output tokens: 638]

Puedes encontrar herramientas en oferta en línea en sitios como Amazon.com.mx, MercadoLibre, The Home Depot México, Mercado Libre Mexico y Suministrointec, con descuentos de hasta 58% y envíos gratis.


In [8]:
result = agent.run(
    "¿Qué material necesito y cuánta cantidad para pintar puertas de mi casa?",
)

print(result)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ ¿Qué material necesito y cuánta cantidad para pintar puertas de mi casa?                                        │
│                                                                                                                 │
╰─ HfApiModel - meta-llama/Llama-3.3-70B-Instruct ────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in code parsing:
Your code snippet is invalid, because the regex pattern ```(?:py|python)?\n(.*?)\n``` was not found in it.
Here is your code snippet:
Para determinar los materiales necesarios y la cantidad para pintar las puertas de tu casa, necesito saber algunos 
detalles más. 

pregunta("¿Cuántas puertas vas a pintar y qué tamaño tienen aproximadamente?")
Make sure to include code with the correct pattern, for instance:
Thoughts: Your thoughts
Code:
```py
# Your python code here
```<end_code>
Make sure to provide correct code blobs.

[Step 0: Duration 3.94 seconds| Input tokens: 470 | Output tokens: 51]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Para pintar 10 puertas de 2x1, necesitarás: \n1. Pintura: 2-3 litros dependiendo del tipo y        
  marca. \n2. Rodillo o pincel: 1-2 unidades. \n3. Trapo: 1-2 unidades. \n4. Guantes: 1 par. \n5. Maskara: 1       
  unidad. \n6. Lija (opcional): 1 unidad.")                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer: Para pintar 10 puertas de 2x1, necesitarás: 
1. Pintura: 2-3 litros dependiendo del tipo y marca. 
2. Rodillo o pincel: 1-2 unidades. 
3. Trapo: 1-2 unidades. 
4. Guantes: 1 par. 
5. Maskara: 1 unidad. 
6. Lija (opcional): 1 unidad.

[Step 2: Duration 9.18 seconds| Input tokens: 2,001 | Output tokens: 264]

Para pintar 10 puertas de 2x1, necesitarás: 
1. Pintura: 2-3 litros dependiendo del tipo y marca. 
2. Rodillo o pincel: 1-2 unidades. 
3. Trapo: 1-2 unidades. 
4. Guantes: 1 par. 
5. Maskara: 1 unidad. 
6. Lija (opcional): 1 unidad.


In [9]:
import pprint

pprint.pprint(agent.memory.steps)

[TaskStep(task='¿Qué material necesito y cuánta cantidad para pintar puertas '
               'de mi casa?',
          task_images=None),
 ActionStep(model_input_messages=[{'content': [{'text': '\n'
                                                        'Eres un asistente '
                                                        'experto en ferretería '
                                                        'que responde de '
                                                        'manera directa y '
                                                        'eficiente solamente a '
                                                        'temas de ferretería y '
                                                        'a ningún otro tema.\n'
                                                        '\n'
                                                        '## Sigue estas '
                                                        'reglas:\n'
                                               